# Face Recognition Experiments using CNN

Using a variety of data sources, attempt to detect facial features and identify subjects.  We attemp to use pre-weighted VGG16 from Keras.

http://www.cl.cam.ac.uk/research/dtg/attarchive/facedatabase.html

http://www.face-rec.org/databases/


In [7]:
%matplotlib inline
import os
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout, Flatten, Reshape
from keras.models import Model, Sequential
import numpy as np
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, Flatten, Lambda, Dropout
from keras.models import Model, load_model
from keras.layers.normalization import BatchNormalization
from keras import optimizers
from keras.applications import vgg16
from matplotlib import pyplot as plt
import numpy as np

In [77]:
# Model to detect if the person has glasses or not
input_shape = (1, 100, 100)
model = Sequential(name='encoder')
model.add(Conv2D(32, 3, 3, input_shape=input_shape, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
model.add(Conv2D(32, 3, 3, activation='relu', border_mode='same'))
model.add(Dropout(0.25))
#model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
model.add(Conv2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
model.add(Conv2D(64, 3, 3, activation='relu', border_mode='same'))
model.add(Dropout(0.25))
model.add(Flatten())
#model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.summary()
#model.load_weights('models/convolutional_recognizer_2.weights.h5')


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_45 (Convolution2D) (None, 32, 100, 100)  320         convolution2d_input_8[0][0]      
____________________________________________________________________________________________________
maxpooling2d_25 (MaxPooling2D)   (None, 32, 50, 50)    0           convolution2d_45[0][0]           
____________________________________________________________________________________________________
convolution2d_46 (Convolution2D) (None, 32, 50, 50)    9248        maxpooling2d_25[0][0]            
____________________________________________________________________________________________________
dropout_23 (Dropout)             (None, 32, 50, 50)    0           convolution2d_46[0][0]           
___________________________________________________________________________________________

In [78]:
model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

In [95]:
def get_batches(path, gen=ImageDataGenerator(height_shift_range=10, horizontal_flip=True), shuffle=True, batch_size=8, class_mode='categorical'):
        return gen.flow_from_directory(path, target_size=(100,100), color_mode='grayscale',
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)
    
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none', cmap='jet')    

In [96]:
batches = get_batches("yalefaces/features/spectacles", batch_size=30)
images, labels = batches.next()

Found 30 images belonging to 2 classes.


In [99]:
nb_epoch = 10
model.fit(images/255., labels, nb_epoch=nb_epoch, verbose=2)

Epoch 1/10
1s - loss: 0.4503 - acc: 0.7667
Epoch 2/10
1s - loss: 0.4330 - acc: 0.7667
Epoch 3/10
1s - loss: 0.4447 - acc: 0.7667
Epoch 4/10
1s - loss: 0.4470 - acc: 0.7667
Epoch 5/10
1s - loss: 0.4380 - acc: 0.7667
Epoch 6/10
1s - loss: 0.4681 - acc: 0.7667
Epoch 7/10
1s - loss: 0.4616 - acc: 0.7667
Epoch 8/10
1s - loss: 0.4535 - acc: 0.7667
Epoch 9/10
1s - loss: 0.4523 - acc: 0.7667
Epoch 10/10
1s - loss: 0.4508 - acc: 0.7667


In [82]:
# Model for recognizing a set of 40 individuals
input_shape = (1, 100, 100)
faces_model = Sequential()
faces_model.add(Conv2D(32, 3, 3, input_shape=input_shape, activation='relu', border_mode='same'))
faces_model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
faces_model.add(Conv2D(32, 3, 3, activation='relu', border_mode='same'))
faces_model.add(Dropout(0.25))
#model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
faces_model.add(Conv2D(64, 3, 3, activation='relu', border_mode='same'))
faces_model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
faces_model.add(Conv2D(64, 3, 3, activation='relu', border_mode='same'))
faces_model.add(MaxPooling2D(pool_size=(2, 2), border_mode='same'))
faces_model.add(Dropout(0.25))
faces_model.add(Flatten())
#model.add(Dense(128, activation='relu'))
faces_model.add(Dense(40, activation='softmax'))
faces_model.summary()
#model.load_weights('models/convolutional_recognizer_2.weights.h5')
faces_model.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_49 (Convolution2D) (None, 32, 100, 100)  320         convolution2d_input_9[0][0]      
____________________________________________________________________________________________________
maxpooling2d_27 (MaxPooling2D)   (None, 32, 50, 50)    0           convolution2d_49[0][0]           
____________________________________________________________________________________________________
convolution2d_50 (Convolution2D) (None, 32, 50, 50)    9248        maxpooling2d_27[0][0]            
____________________________________________________________________________________________________
dropout_25 (Dropout)             (None, 32, 50, 50)    0           convolution2d_50[0][0]           
___________________________________________________________________________________________

In [106]:
batches = get_batches("att_faces", batch_size=400)
test_batches = get_batches("att_faces_test", batch_size=40)
nb_epoch = 10
nb_val_samples=test_batches.N
X, Y = batches.next()
faces_model.fit(X/255., Y, nb_epoch=nb_epoch, verbose=2)
X_test, Y_test = test_batches.next()
loss, acc = faces_model.test_on_batch(X_test/255., Y_test)
print("Loss: %.5f Accuracy: %.5f" % (loss, acc))

Found 400 images belonging to 40 classes.
Found 80 images belonging to 40 classes.
Epoch 1/10
9s - loss: 0.1049 - acc: 0.9754
Epoch 2/10
9s - loss: 0.1019 - acc: 0.9757
Epoch 3/10
9s - loss: 0.0989 - acc: 0.9755
Epoch 4/10
8s - loss: 0.0967 - acc: 0.9755
Epoch 5/10
8s - loss: 0.0961 - acc: 0.9759
Epoch 6/10
9s - loss: 0.0938 - acc: 0.9762
Epoch 7/10
8s - loss: 0.0909 - acc: 0.9763
Epoch 8/10
9s - loss: 0.0891 - acc: 0.9769
Epoch 9/10
9s - loss: 0.0861 - acc: 0.9766
Epoch 10/10
9s - loss: 0.0847 - acc: 0.9771
Loss: 0.08386 Accuracy: 0.97625
